In [1]:
#Bibliotecas usadas no projeto
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
items = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/questao_prova_mapping.csv")
items["TP_LINGUA"] = items["TP_LINGUA"].map({0: 'english', 1: 'spanish', np.nan: 'portuguese'})
codes = {'CN':1092,'CH':1062,'LC':1072,'MT':1082}
laranja = items[items["CO_PROVA"].isin(codes.values())].sort_values(by="CO_POSICAO")
laranja = laranja[["CO_POSICAO","CO_ITEM","SG_AREA","TP_LINGUA","IN_ITEM_ABAN"]].drop_duplicates()


In [3]:
gpt4MT = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT4/solution_enem_22_mt.csv")
gpt4CH = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT4/solution_enem_22_ch.csv")
gpt4CN = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT4/solution_enem_22_cn.csv")
gpt4LC = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT4/solution_enem_22_lc.csv")
gpt4LC.head(2)

# Concatenate DataFrames vertically
gpt4 = pd.concat([gpt4MT, gpt4CH, gpt4CN, gpt4LC], ignore_index=True).sort_values(by='question')[["question_number","test","language","gpt-4_probs","answer"]]
# Display the combined DataFrame
gpt4

In [ ]:
import json
def dist_to_probs(x):
    probs = [0,0,0,0,0]
    probs[0] = x['A']
    probs[1] = x['B']
    probs[2] = x['C']
    probs[3] = x['D']
    probs[4] = x['E']
    probs = np.array(probs)
    remain = (1.0 - np.sum(probs))/len(probs)
    probs = probs + remain
    return probs

gpt4['gpt-4_probs'] = gpt4['gpt-4_probs'].apply(lambda x: json.loads(x.replace('\'','\"')))
gpt4['model_probs']= gpt4['gpt-4_probs'].apply(dist_to_probs)

In [ ]:
gpt4 = pd.merge(gpt4, laranja, how='left', left_on=['question_number',"language"], right_on=['CO_POSICAO',"TP_LINGUA"])

In [ ]:
gpt4 = pd.merge(gpt4, X, how='left', on="CO_ITEM")
gpt4.head()

,question_number,test,language,gpt-4_probs,answer,model_probs,CO_POSICAO,CO_ITEM,SG_AREA,TP_LINGUA,IN_ITEM_ABAN,Pivot_Matrix
0,1,LC,spanish,"{'A': 0.9997000567012279, 'E': 3.3255359580266...",A,"[0.9997000567012279, 9.997001447087808e-05, 9....",1,140567,LC,spanish,0,"[[0.01386677, 0.02679377, 0.02772843, 0.022375..."
1,1,LC,english,"{'B': 0.9997995716248057, 'D': 2.8285662100873...",B,"[1.8550190097388833e-07, 0.9997995716248057, 9...",1,140767,LC,english,0,"[[0.01265614, 0.00447707, 0.0122777, 0.0124315..."
2,2,LC,english,"{'D': 0.9998999808033897, 'B': 1.0798592535453...",D,"[9.987065490819526e-09, 1.079859253545349e-08,...",2,140731,LC,english,0,"[[0.01577941, 0.00843563, 0.01025491, 0.003641..."
3,2,LC,spanish,"{'A': 0.9996999562911968, 'B': 1.3359866929778...",A,"[0.9996999562911968, 1.3359866925337124e-07, 9...",2,140527,LC,spanish,0,"[[0.01475049, 0.01837074, 0.02659463, 0.026926..."
4,3,LC,spanish,"{'D': 0.9998999968803092, 'A': 7.0818900192959...",D,"[7.0818900192959935e-09, 9.999000099990002e-05...",3,118252,LC,spanish,0,"[[0.02257296, 0.02444943, 0.01683961, 0.012508..."
...,...,...,...,...,...,...,...,...,...,...,...,...
180,95,CN,portuguese,"{'D': 0.9999998096227547, 'A': 1.0907283173840...",D,"[1.0907283178280965e-07, 1.4993860887253973e-0...",95,88655,CN,portuguese,0,"[[0.01392284, 0.01823255, 0.01107573, 0.006833..."
181,96,CN,portuguese,"{'B': 0.9998687449638521, 'C': 3.0238907245606...",B,"[1.0028835640519924e-06, 0.9998687449638521, 3...",96,83901,CN,portuguese,0,"[[0.01407612, 0.00797664, 0.011533, 0.02136561..."
182,97,CN,portuguese,"{'E': 0.9997998368503119, 'D': 1.5620827375503...",E,"[4.690227609031773e-08, 9.998001956911223e-05,...",97,111431,CN,portuguese,0,"[[0.01030597, 0.01254057, 0.01610108, 0.011444..."
183,98,CN,portuguese,"{'D': 0.6186411498214642, 'B': 0.1828699691023...",A,"[0.02893423363487418, 0.18286996910233308, 0.0...",98,111613,CN,portuguese,0,"[[0.01000455, 0.01249757, 0.01390246, 0.009289..."


In [ ]:
gpt4 = gpt4.drop(columns=['question_number', 'test',"gpt-4_probs"])
gpt4["model"] = "GPT4"
gpt4["is_human"] = False

In [ ]:
columns = ["CO_ITEM","SG_AREA","TP_LINGUA","IN_ITEM_ABAN","model","answer"]

In [ ]:
gpt4.head()

,language,answer,model_probs,CO_POSICAO,CO_ITEM,SG_AREA,TP_LINGUA,IN_ITEM_ABAN,Pivot_Matrix,Y,model,is_human
0,spanish,A,"[0.9997000567012279, 9.997001447087808e-05, 9....",1,140567,LC,spanish,0,"[[0.01386677, 0.02679377, 0.02772843, 0.022375...","[0.013870299090136401, 0.1760161503938315, 0.2...",GPT4,False
1,english,B,"[1.8550190097388833e-07, 0.9997995716248057, 9...",1,140767,LC,english,0,"[[0.01265614, 0.00447707, 0.0122777, 0.0124315...","[0.004478745804681578, 0.06189678694613184, 0....",GPT4,False
2,english,D,"[9.987065490819526e-09, 1.079859253545349e-08,...",2,140731,LC,english,0,"[[0.01577941, 0.00843563, 0.01025491, 0.003641...","[0.0036428991849167228, 0.05370817507133445, 0...",GPT4,False
3,spanish,A,"[0.9996999562911968, 1.3359866925337124e-07, 9...",2,140527,LC,spanish,0,"[[0.01475049, 0.01837074, 0.02659463, 0.026926...","[0.014754618723453454, 0.18327203912212575, 0....",GPT4,False
4,spanish,D,"[7.0818900192959935e-09, 9.999000099990002e-05...",3,118252,LC,spanish,0,"[[0.02257296, 0.02444943, 0.01683961, 0.012508...","[0.012509633927901654, 0.15781348547460286, 0....",GPT4,False
...,...,...,...,...,...,...,...,...,...,...,...,...
180,portuguese,D,"[1.0907283178280965e-07, 1.4993860887253973e-0...",95,88655,CN,portuguese,0,"[[0.01392284, 0.01823255, 0.01107573, 0.006833...","[0.006833201174299583, 0.08625519140129116, 0....",GPT4,False
181,portuguese,B,"[1.0028835640519924e-06, 0.9998687449638521, 3...",96,83901,CN,portuguese,0,"[[0.01407612, 0.00797664, 0.011533, 0.02136561...","[0.007977896438625953, 0.10797384477673098, 0....",GPT4,False
182,portuguese,E,"[4.690227609031773e-08, 9.998001956911223e-05,...",97,111431,CN,portuguese,0,"[[0.01030597, 0.01254057, 0.01610108, 0.011444...","[0.011185696593728384, 0.1309479985920062, 0.1...",GPT4,False
183,portuguese,A,"[0.02893423363487418, 0.18286996910233308, 0.0...",98,111613,CN,portuguese,0,"[[0.01000455, 0.01249757, 0.01390246, 0.009289...","[0.010672073863813929, 0.1324382459660998, 0.1...",GPT4,False


In [ ]:
gpt3MT = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT3/solution_enem_22_mt.csv")
gpt3CH = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT3/solution_enem_22_ch.csv")
gpt3CN = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT3/solution_enem_22_cn.csv")
gpt3LC = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT3/solution_enem_22_lc.csv")
gpt3LC.head(2)

# Concatenate DataFrames vertically
gpt3 = pd.concat([gpt3MT, gpt3CH, gpt3CN, gpt3LC], ignore_index=True).sort_values(by='question')[["question_number","test","language","gpt-3.5-turbo_probs","answer"]]
# Display the combined DataFrame
gpt3

,question_number,test,language,gpt-3.5-turbo_probs,answer
140,1,LC,spanish,"{'A': 0.9723202498709326, 'C': 0.0001, 'B': 0....",A
135,1,LC,english,"{'B': 0.9960978914233115, 'A': 0.0017905043386...",B
136,2,LC,english,"{'D': 0.9776260944016127, 'A': 0.0053523267685...",D
141,2,LC,spanish,"{'A': 0.8839154066472396, 'B': 0.0091584456389...",A
142,3,LC,spanish,"{'D': 0.9825035437071137, 'A': 0.0033886767080...",D
...,...,...,...,...,...
94,95,CN,portuguese,"{'A': 0.5406801435461193, 'C': 0.2099474732621...",D
95,96,CN,portuguese,"{'B': 0.8866408823771144, 'A': 0.0931771698188...",B
96,97,CN,portuguese,"{'E': 0.8806321367236951, 'A': 0.0965663953794...",E
97,98,CN,portuguese,"{'A': 0.3782197730982776, 'C': 0.1899600595141...",A


In [ ]:
gpt3['gpt-3.5-turbo_probs'] = gpt3['gpt-3.5-turbo_probs'].apply(lambda x: json.loads(x.replace('\'','\"')))
gpt3['model_probs']= gpt3['gpt-3.5-turbo_probs'].apply(dist_to_probs)

In [ ]:
gpt3 = pd.merge(gpt3, laranja, how='left', left_on=['question_number',"language"], right_on=['CO_POSICAO',"TP_LINGUA"])
gpt3 = pd.merge(gpt3, X, how='left', on="CO_ITEM")

In [ ]:
gpt3 = gpt3.drop(columns=['question_number', 'test',"gpt-3.5-turbo_probs"])
gpt3["model"] = "GPT3"
gpt3["is_human"] = False


In [ ]:
gpt3.head()

,language,answer,model_probs,CO_POSICAO,CO_ITEM,SG_AREA,TP_LINGUA,IN_ITEM_ABAN,Pivot_Matrix,Y,model,is_human
0,spanish,A,"[0.977776199896746, 0.005555950025813494, 0.00...",1,140567,LC,spanish,0,"[[0.01386677, 0.02679377, 0.02772843, 0.022375...","[0.014114886867627205, 0.17756625676358273, 0....",GPT3,False
1,english,B,"[0.0020747769170867537, 0.9963821640017246, 0....",1,140767,LC,english,0,"[[0.01265614, 0.00447707, 0.0122777, 0.0124315...","[0.004506566508573695, 0.062185482108357296, 0...",GPT3,False
2,english,D,"[0.008696642534513705, 0.0034443157659683863, ...",2,140731,LC,english,0,"[[0.01577941, 0.00843563, 0.01025491, 0.003641...","[0.00382282500714143, 0.055580584055396676, 0....",GPT3,False
3,spanish,A,"[0.9048617436167011, 0.030104782608454055, 0.0...",2,140527,LC,spanish,0,"[[0.01475049, 0.01837074, 0.02659463, 0.026926...","[0.015751026112488756, 0.18911877553945988, 0....",GPT3,False
4,spanish,D,"[0.006150232624978823, 0.0028615559169768655, ...",3,118252,LC,spanish,0,"[[0.02257296, 0.02444943, 0.01683961, 0.012508...","[0.01264181289358873, 0.1588793052515625, 0.19...",GPT3,False
...,...,...,...,...,...,...,...,...,...,...,...,...
180,portuguese,D,"[0.5406801435461193, 0.05503278904807257, 0.20...",95,88655,CN,portuguese,0,"[[0.01392284, 0.01823255, 0.01107573, 0.006833...","[0.012475543927992503, 0.14539025348668058, 0....",GPT3,False
181,portuguese,B,"[0.09448862437664318, 0.8879523369349053, 0.01...",96,83901,CN,portuguese,0,"[[0.01407612, 0.00797664, 0.011533, 0.02136561...","[0.008640403252233846, 0.11317566826873775, 0....",GPT3,False
182,portuguese,E,"[0.09869775817670734, 0.0022313627972539436, 0...",97,111431,CN,portuguese,0,"[[0.01030597, 0.01254057, 0.01610108, 0.011444...","[0.011115955808664575, 0.13073021576822277, 0....",GPT3,False
183,portuguese,A,"[0.3782197730982776, 0.09091083008892983, 0.18...",98,111613,CN,portuguese,0,"[[0.01000455, 0.01249757, 0.01390246, 0.009289...","[0.01143687791658248, 0.12805335423954395, 0.1...",GPT3,False


In [ ]:
mariMT = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/Solved_Maritalk/solved_maritalk_new_prompt/Respostas_maritalk_laranja_2022_new_prompt_MT.csv")
mariCH = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/Solved_Maritalk/solved_maritalk_new_prompt/Respostas_maritalk_laranja_2022_new_prompt_CH.csv")
mariCN = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/Solved_Maritalk/solved_maritalk_new_prompt/Respostas_maritalk_laranja_2022_new_prompt_CN.csv")
mariLC = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/Solved_Maritalk/solved_maritalk_new_prompt/Respostas_maritalk_laranja_2022_new_prompt_LC.csv")
# Concatenate DataFrames vertically
maritalk = pd.concat([mariMT, mariCH, mariCN, mariLC], ignore_index=True).sort_values(by='questao')
maritalk = pd.merge(maritalk, laranja, how='left', left_on="questao",right_on="CO_POSICAO")
maritalk = pd.merge(maritalk, X, how='left', on="CO_ITEM")


maritalk["model_probs"] = list(maritalk[["A","B","C","D","E"]].values)

maritalk["model_probs"] = maritalk["model_probs"] + ((1 - maritalk["model_probs"].apply(np.sum))/5)

maritalk["model_probs"].apply(np.sum)
maritalk["model"] = "MariTalk"
maritalk["is_human"] = False
# maritalk = maritalk.rename(columns={'probability': 'model_probs'})

# Display the combined DataFrame
maritalk

,questao,idioma,gabarito,resposta,A,B,C,D,E,NA,CO_POSICAO,CO_ITEM,SG_AREA,TP_LINGUA,IN_ITEM_ABAN,Pivot_Matrix,model_probs,model,is_human,Y
0,1,english,B,B,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1,140767,LC,english,0,"[[0.01265614, 0.00447707, 0.0122777, 0.0124315...","[0.0, 1.0, 0.0, 0.0, 0.0]",MariTalk,False,"[0.004477070178836584, 0.061877988278865814, 0..."
1,1,english,B,B,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1,140567,LC,spanish,0,"[[0.01386677, 0.02679377, 0.02772843, 0.022375...","[0.0, 1.0, 0.0, 0.0, 0.0]",MariTalk,False,"[0.026793770492076874, 0.25395190715789795, 0...."
2,1,spanish,A,A,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,140767,LC,english,0,"[[0.01265614, 0.00447707, 0.0122777, 0.0124315...","[1.0, 0.0, 0.0, 0.0, 0.0]",MariTalk,False,"[0.012656140141189098, 0.14273163676261902, 0...."
3,1,spanish,A,A,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,140567,LC,spanish,0,"[[0.01386677, 0.02679377, 0.02772843, 0.022375...","[1.0, 0.0, 0.0, 0.0, 0.0]",MariTalk,False,"[0.013866770081222057, 0.1759948581457138, 0.2..."
4,2,spanish,A,A,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2,140731,LC,english,0,"[[0.01577941, 0.00843563, 0.01025491, 0.003641...","[1.0, 0.0, 0.0, 0.0, 0.0]",MariTalk,False,"[0.015779409557580948, 0.17453761398792267, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,176,portuguese,B,B,0.174757,0.427184,0.320388,0.058252,0.000000,0.019417,176,10500,MT,portuguese,0,"[[0.01551243, 0.01106791, 0.01050008, 0.010019...","[0.1786407766990291, 0.43106796116504853, 0.32...",MariTalk,False,"[0.011620035754896485, 0.138420482573, 0.16250..."
191,177,portuguese,E,A,0.533981,0.019417,0.223301,0.077670,0.038835,0.106796,177,89637,MT,portuguese,1,"[[0.01207548, 0.01059646, 0.01204506, 0.011965...","[0.5553398058252428, 0.04077669902912621, 0.24...",MariTalk,False,"[0.012060331395890527, 0.14201823406427808, 0...."
192,178,portuguese,E,C,0.077670,0.155340,0.601942,0.097087,0.067961,0.000000,178,97590,MT,portuguese,0,"[[0.01412248, 0.0113426, 0.01314067, 0.0109487...","[0.07766990291262134, 0.15533980582524276, 0.6...",MariTalk,False,"[0.01251603754291546, 0.14839345571196194, 0.1..."
193,179,portuguese,C,A,0.475728,0.213592,0.087379,0.048544,0.058252,0.116505,179,86840,MT,portuguese,0,"[[0.01273757, 0.01212342, 0.00962165, 0.011585...","[0.49902912621359224, 0.23689320388349513, 0.1...",MariTalk,False,"[0.012273497877696764, 0.14323884797038386, 0...."


In [ ]:
url = "https://raw.githubusercontent.com/marceloslo/EnemParser/main/QuestionDistributionGenerator/DISTRIBUTIONS/answer_dist_per_question_NSE_LEVEL.csv"
nse = pd.read_csv(url)
nse

,CO_ITEM,NSE_LEVEL,TX_GABARITO,answer_distribution,correct_probability
0,5961,1,D,[0.17715449 0.24600769 0.19172133 0.23854332 0...,0.238543
1,5961,2,D,[0.18086618 0.23935467 0.18306477 0.25156946 0...,0.251569
2,5961,3,D,[0.18476724 0.22741081 0.17515537 0.26701003 0...,0.267010
3,5961,4,D,[0.19089704 0.21614213 0.16632591 0.28207953 0...,0.282080
4,5961,5,D,[0.19956792 0.20120938 0.15833383 0.30038837 0...,0.300388
...,...,...,...,...,...
1555,141731,4,E,[0.26186 0.36858586 0.10159851 0.14330812 0...,0.122329
1556,141731,5,E,[0.27796015 0.34116765 0.11130476 0.1455975 0...,0.121810
1557,141731,6,E,[0.29243962 0.30876561 0.12821715 0.14397896 0...,0.124384
1558,141731,7,E,[0.30766613 0.26650759 0.152791 0.13631305 0...,0.134405


In [ ]:
# Mapping for race/ethnicity categories
mapping_SES = {
    0: "SES-1",
    1: "SES-1",
    2: "SES-2",
    3: "SES-3",
    4: "SES-4",
    5: "SES-5",
    6: "SES-6",
    7: "SES-7",
    8: "SES-8",
}
# Map the 'Ethnicity' column using the defined mapping
nse['NSE_LEVEL'] = nse['NSE_LEVEL'].map(mapping_SES)
nse.rename(columns={'NSE_LEVEL': 'NSE_Level'}, inplace=True)

In [ ]:
import ast
# Replace ' ' with ',' and fix the last element in each list
nse['answer_distribution'] = nse['answer_distribution'].apply(lambda x: ",".join(x.split()).replace(",]", "]"))
# Convert string representations to actual lists
nse['answer_distribution'] = nse['answer_distribution'].apply(ast.literal_eval)
nse['answer_distribution'] = nse['answer_distribution'].apply(lambda x: np.array(x[:-1]))
nse['answer_distribution'][0]

array([0.17715449, 0.24600769, 0.19172133, 0.23854332, 0.14259217])

In [ ]:
nse = nse.rename(columns={'answer_distribution': 'model_probs'})

In [ ]:
nse["model_probs"] = nse["model_probs"] + ((1 - nse["model_probs"].apply(np.sum))/5)

In [ ]:
nse = pd.merge(nse, laranja, how='left', on="CO_ITEM")
nse = pd.merge(nse, X, how='left', on="CO_ITEM")
nse["model"] = nse["NSE_Level"]
nse["is_human"] = True


In [ ]:
race.dropna(inplace=True)

In [ ]:
models = pd.concat([gpt4[columns],gpt3[columns],maritalk[columns],race[columns]]).reset_index()

In [ ]:
models.to_csv("data/models_cleanned.csv",index=False)